# Speaker metric studies : 


In [84]:
import json

In [85]:
refPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/reference_transcript_to_evaluate.json"
hypPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/vocapia/nlp_transcript_norm.json"

In [86]:
#method to read a transcript json file
def read_transcript(transcriptPath):
    try:
        return json.load(open(transcriptPath,'rt',encoding='utf-8'))
    except FileNotFoundError:
        return None

# Study 1
## Table de correspondances des speaker avec leurs intervals de temps de parole

In [87]:
#method to get the speaker time intervals
def get_speaker_screen_time(transcript):
    speakers = transcript['speakers']
    words = transcript['words']
    speakerScreenTimeList = []
    
    for speaker in speakers : 

        speakerId = speaker['id']
        speakerScreenTime =[]

        interList = []
        i = 0
        while i < len(words):
            interList = []
            while i < len(words) and words[i]['speakerId'] == speakerId:
                interList.append(words[i])
                i+=1
            
            if len(interList) > 0 :
                screenTime = {
                    "startTime" : interList[0]['startTime'],
                    "endTime" : interList[-1]['endTime']
                }
                speakerScreenTime.append(screenTime)
                interList = []
            else :
                i+=1

        
        speaker['speakerScreenTime'] = speakerScreenTime
        speakerScreenTimeList.append(speaker)
                    

    return speakerScreenTimeList
           

In [88]:
#method to compare two tables of speaker screentimes
#TO DO 

# Study 2 : 
## Detection des changements de speaker 

In [89]:
#method to get the exact time where a change of speaker is detected regardless of the speaker id
def get_speaker_change_detection(transcript):
    
    words = transcript['words']
    speakerChangeTimes = []
    
    i = 0
    actualSpeaker = words[0]['speakerId']
    
    while i < len(words) :
        if actualSpeaker != words[i]['speakerId']:
            speakerChangeTimes.append(words[i]['startTime'])
            actualSpeaker = words[i]['speakerId']
            
        i+=1
    
    return speakerChangeTimes
    

In [166]:
#method to compare two speaker change time
#TO DO
def compare_speaker_changes(refSpeakerChangeTimes, hypSpeakerChangeTimes):
    confTime = float(0.5)
    
    matches = 0
    
    for refChangeTime in refSpeakerChangeTimes : 
        for hypChangeTime in hypSpeakerChangeTimes :
            
            if float(refChangeTime) >= float(hypChangeTime - confTime) and float(refChangeTime) >= float(hypChangeTime + confTime):
                matches += 1
                hypSpeakerChangeTimes.remove(hypChangeTime)
                break 

                
    result = {
        "matching_changes" : matches,
        "unexpected_changes" : len(hypSpeakerChangeTimes),
        "missed_changes" : len(refSpeakerChangeTimes) - matches
    }
        
    errorRate = (result['unexpected_changes'] + result['missed_changes']) / len(refSpeakerChangeTimes)
    return errorRate
    
    
    
    

# TEST AREA

In [167]:
hypTranscript = read_transcript(hypPath)
refTranscript = read_transcript(refPath)

In [168]:
hypScreenTime = get_speaker_screen_time(hypTranscript)
refScreenTime = get_speaker_screen_time(refTranscript)

In [169]:
hypSpeakerChangeTimes = get_speaker_change_detection(hypTranscript)
refSpeakerChangeTimes = get_speaker_change_detection(refTranscript)

In [171]:
print(compare_speaker_changes(refSpeakerChangeTimes, hypSpeakerChangeTimes))

0.3253012048192771
